# Custom Report Generator
### Takes in tNGS outfile and sequencing load file to create 'custom report' file to be able to import into Starlims

In [1]:
import pandas as pd
import os

In [2]:
tngs = pd.read_csv('../output/tNGS_export.csv')
seq = pd.read_csv('../data/seq1.csv',delimiter="\t")
tngs.columns = [c.replace(' ', '_') for c in tngs.columns]
seq.columns = [c.replace(' ', '_') for c in seq.columns]

In [4]:
# Starlims sequencing'NGS' workbatch number
workbatchNo = seq['Container_Name'][0]

In [5]:
# Get starlims 'amplicon'
star_amplicon = "_NGS_"

In [6]:
# need to get a dictionary of all sample ID from sequencing load file as that represents what can get 
# imported back into Starlims. Mostly important for the ddPCR mutation details import
samples = list(seq['Plate_ID'].unique())
del samples[:4]
samples = [sample[:9] for sample in samples]

In [7]:
# new column with sample id and mutsurveyor variant
tngs['id_variant'] = tngs['Folder_number'] + "_" + tngs['MutSurveyor']
tngs_var = list(tngs['id_variant'].unique())
tngs_var = [var for var in tngs_var if str(var) != 'nan']

In [8]:
# create a dictionary with sample id's and associated variants
sample_vars = {k:[] for k in samples}
for sample in samples:
    for variant in tngs_var:
        if sample == variant[:9]:
            try:
                sample_vars[sample].append(variant[10:])
            except:
                sample_vars[sample] = [variant[10:]]

In [10]:
# create custom report template
def create_custom_report():
    f = open(f'{os.getcwd()}/../output/custom_report.txt', 'w+')
    header = "Warning!\nSample Name\tReference Name\tLane Quality\tROI Coverage\t#nts below threshold\tQuality ROI\tVariant1\tVariant3\tVariant3\tVariant4\n"
    body = ""
    for sample in sample_vars:
        no_vars = len(sample_vars[sample])
        sample_id = sample + star_amplicon + workbatchNo
        if no_vars == 1:
            body += f"{sample_id}\t\t\t\t\t\t{sample_vars[sample][0]}\n"
        elif no_vars == 2:
            body += f"{sample_id}\t\t\t\t\t\t{sample_vars[sample][0]}\t{sample_vars[sample][1]}\n"        
        elif no_vars == 3:
            body += f"{sample_id}\t\t\t\t\t\t{sample_vars[sample][0]}\t{sample_vars[sample][1]}\t{sample_vars[sample][2]}\n" 
        elif no_vars == 4:
            body += f"{sample_id}\t\t\t\t\t\t{sample_vars[sample][0]}\t{sample_vars[sample][1]}\t{sample_vars[sample][2]}\t{sample_vars[sample][3]}\n"
        else:
            body += f"{sample_id}\t\t\t\t\t\t\t\t\t\t\n"
    f.write(header)
    f.write(body)
    f.close()

create_custom_report()

In [ ]:
# create mutation details 'ddPCR' import file

def create_mut_details():
    pass
